In [30]:
pip install streamlit -q

In [31]:
pip install Pillow

In [32]:
!wget -q -O - ipv4.icanhazip.com

34.45.96.9


In [72]:
%%writefile app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

st.title("Algoritmo 3: ")
st.header("Random Forest")
st.write("Este algoritmo usa las métricas de MAE y R2 para realizar sus predicciones. Los datos fueron obtenidos del archivo 'laptop_price.csv. Las caracteristicas analizadas fueron: 'Inches', 'Ram', 'Weight', 'Cpu_hgz', 'IPS', 'Touchscreen', 'screen_width', 'SSD_GB','HDD_GB' ")

#datos
df = pd.read_csv('laptop_price.csv', encoding='ISO-8859-1')

#Borrado de laptop ID al no tener informacion valiada solo es campo de numeracion correlativa
df.drop(columns=['laptop_ID'],inplace=True)

# convertir a numerico: la columna Ram
df['Ram']=df['Ram'].str.replace('GB','').astype(float)

# Ubicamosla columna que puede ser usada para OHE y aplicamos OHE
# La columna identificada para OHE es tipo de laptop ('TypeName')
df['TypeName'].value_counts()
df1=pd.get_dummies(df,columns=['TypeName'],drop_first=True ,dtype=int)

# Columna Weight: Extraemos los valores numéricos de la columna Weight
df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)

# Columna Cpu_GHz: Extraemos la velocidad del procesador y generamos una coluna nueva
df['Cpu_hgz'] = df['Cpu'].apply(lambda x: x.split()[-1].replace("GHz", "")).astype(float)

# Extraendo la marca/modelo de procesaros
def fetch_processor(x):
  cpu_name = " ".join(x.split()[0:3])
  if cpu_name == 'Intel Core i3' or cpu_name == 'Intel Core i5' or cpu_name == 'Intel Core i7':
    return cpu_name
  elif cpu_name.split()[0] == 'Intel':
    return 'Otro modelo Intel'
  else:
    return 'Procesador AMD'

df['Modelo_Cpu'] = df['Cpu'].apply(lambda x: fetch_processor(x))
#Procesado la columna Cpu procedemos al borado
df.drop(columns=['Cpu'],inplace=True)

# Creamos 2 columnas si es IPS o Touchscreen
df['IPS']=df['ScreenResolution'].apply(lambda x:1 if 'IPS' in x else 0)
df['Touchscreen'] = df['ScreenResolution'].apply(lambda x:1 if 'Touchscreen' in x else 0)

# Extraemos el las dimexiones de la pantalla X,Y

df[['screen_width', 'screen_height']] = df['ScreenResolution'].str.extract(r'(\d{3,4})x(\d{3,4})').astype(float)
df.head()

#Limpieza de columans procesadas
df.drop(columns=['ScreenResolution'],inplace=True)

# Procesamos la columna memoria
# Generamos nuevas columnas de tipo de memoria con el tamaño del mismo

#-------------------------------------------------------------------------------
#mean_absolute_error = st.empty()
#r2_score = st.empty()

import re
# Función para convertir la memoria a gigabytes (GB) y clasificar por tipo
def extract_memory_types(memory_string):
    # Inicializar valores
    ssd_gb = 0
    hdd_gb = 0
    flash_gb = 0
    hybrid_gb = 0

    # Dividir la cadena en componentes si hay '+'
    components = re.split(r'\s*\+\s*', memory_string)

    for component in components:
        # Buscar todas las capacidades y tipos de almacenamiento
        matches = re.findall(r'(\d+(\.\d+)?)\s*(GB|TB|MB)?\s*(SSD|HDD|Flash Storage|Hybrid)?', component)

        for match in matches:
            value = float(match[0])
            unit = match[2] if match[2] else 'GB'
            storage_type = match[3] if match[3] else 'Unknown'

            # Convertir la unidad a GB
            if unit == 'TB':
                value *= 1024  # 1 TB = 1024 GB
            elif unit == 'MB':
                value /= 1024  # 1 GB = 1024 MB

            # Asignar el valor a la columna correspondiente
            if 'SSD' in storage_type:
                ssd_gb += value
            elif 'HDD' in storage_type:
                hdd_gb += value
            elif 'Flash Storage' in storage_type:
                flash_gb += value
            elif 'Hybrid' in storage_type:
                hybrid_gb += value

    return pd.Series([ssd_gb, hdd_gb, flash_gb, hybrid_gb], index=['SSD_GB', 'HDD_GB', 'Flash_GB', 'Hybrid_GB'])

# Aplicar la función a la columna
df[['SSD_GB', 'HDD_GB', 'Flash_GB', 'Hybrid_GB']] = df['Memory'].apply(extract_memory_types)
# Boramos la columna procesada
df.drop(columns=['Memory'],inplace=True)

# Extraemos la marca de GPU
df['Marca_Gpu'] = df['Gpu'].apply(lambda x:x.split()[0])
df.drop(columns=['Gpu'],inplace=True)
#Extraemos qué sistema OpSys
def os(inp):
    if inp == 'Windows 10' or inp == 'Windows 7' or inp == 'Windows 10 S':
        return 'Windows'
    elif inp == 'macOS' or inp == 'Mac OS X':
        return 'Mac'
    else:
        return 'Otros/No SO/Linux'
df['os'] = df['OpSys'].apply(os)
df.drop(columns=['OpSys'],inplace=True)

#Evaluamos el mapa de calor
#plt.figure(figsize=(15, 9))
#sns.heatmap(df.corr(numeric_only=True), cmap='coolwarm', annot=True)
#plt.show()

#Tarea 3:

#seleccionar las columnas para X que ustedes concideran son las de mas alta correlacion
#seleccionamos las mayores a 0.1
# Crear X , y

X = df[['Inches','Ram','Weight','Cpu_hgz','IPS', 'Touchscreen','screen_width','SSD_GB','HDD_GB' ]]
y = np.log(df['Price_euros'])


# Division de los Datos
# 85% para entrenamiento, 15% para prueba

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

#Función ------------------------------------------------------------------------------------------
def predecir_rfr():

# Estandarizacion
  scaler = StandardScaler()
  X_train_sc = scaler.fit_transform(X_train)
  X_test_sc = scaler.transform(X_test)

# Tarea 4: Random Forest (Bosque Aleatorio) para Regresion

  rfr = RandomForestRegressor(n_estimators=100,random_state=42,max_samples=0.5,max_features=0.75,max_depth=15)
  rfr.fit(X_train_sc, y_train)

# Evaluacion de Algoritmo 3: Evaluacion de Ramdom Forest
  y_pred_rfr = rfr.predict(X_test_sc)
# Hacer predicciones

# Evaluar que tan correctas / Precisas son las predicciones
# IMPORTANTE: en este caso escoger metricas de regresion: MAE, R2 score

  mae_rfr = mean_absolute_error(y_test, y_pred_rfr)
  r2_rfr = r2_score(y_test, y_pred_rfr)

  st.write(f"MAE Random Forest: {mae_rfr:.2f}")
  st.write(f"R2 Score Random Forest: {r2_rfr:.2f}%")

#Crear el botón
if st.button('Predecir'):
  predecir_rfr()



Overwriting app.py


In [73]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.45.96.9:8501

your url is: https://open-memes-wink.loca.lt
  Stopping...
^C
